In [1]:
# Get Layers

import ezdxf

def get_layers_in_dxf(file_path):
    layers = []
    dwg = ezdxf.readfile(file_path)
    for layer in dwg.layers:
        layers.append(layer.dxf.name)
    return layers

def main():
    file_path = "/Users/user/Downloads/test.dxf"  # Update with the path to your DXF file
    layers = get_layers_in_dxf(file_path)
    print("Layers in the DXF file:")
    for layer in layers:
        print(layer)

In [2]:
main()

Layers in the DXF file:
0
A-WALL
A-DOOR
A-GLAZ
A-FLOR
Q-CASE
S-STRS
S-STRS-IDEN
C-PROP-LINE
A-AREA
A-WALL-PATT
A-DETL-GENF
S-STRS-ANNO
A-DETL-THIN
A-FLOR-OVHD
A-AREA-IDEN
A-ANNO-DIMS
A-ANNO-DIMS-100
Defpoints


In [3]:
# Count entities in layer

def count_entities_in_layer(file_path, layer_name):
    entity_count = 0
    dwg = ezdxf.readfile(file_path)
    modelspace = dwg.modelspace()
    for entity in modelspace:
        if entity.dxf.layer == layer_name:
            entity_count += 1
    return entity_count

def main2():
    file_path = "/Users/user/Downloads/Sample1-lines.dxf"  # Update with the path to your DXF file
    layer_name = "A-WALL"  # Specify the layer name you want to count entities for
    entity_count = count_entities_in_layer(file_path, layer_name)
    print(f"Number of entities in layer '{layer_name}': {entity_count}")

In [4]:
main2()

Number of entities in layer 'A-WALL': 24


In [5]:
# Count type of entites in layer

def count_solids_in_layer(file_path, layer_name):
    entitytype_count = 0
    dwg = ezdxf.readfile(file_path)
    modelspace = dwg.modelspace()
    for entity in modelspace:
        if entity.dxftype() == 'LINE' and entity.dxf.layer == layer_name:
            entitytype_count += 1
    return entitytype_count

def main3():
    file_path = "/Users/user/Downloads/Sample2-solids.dxf"  # Update with the path to your DXF file
    layer_name = "A-WALL"  # Specify the layer name you want to count SOLID entities for
    entitytype_count = count_solids_in_layer(file_path, layer_name)
    print(f"Number of LINE entities in layer '{layer_name}': {entitytype_count}")

In [6]:
main3()

Number of LINE entities in layer 'A-WALL': 24


In [3]:
import ezdxf

def get_line_length(line):
    return (line.dxf.end - line.dxf.start).magnitude

def separate_lines(lines, threshold):
    return [line for line in lines if get_line_length(line) > threshold]

def group_Vlines(lines, distance_threshold):
    # Sort the lines by length from longest to shortest
    lines = sorted(lines, key=lambda line: abs(line.dxf.start[1] - line.dxf.end[1]), reverse=True)
    groups = []
    
    while len(lines) > 0:
        current_line = lines.pop(0)
        span_start = min(current_line.dxf.start[1], current_line.dxf.end[1])
        span_end = max(current_line.dxf.start[1], current_line.dxf.end[1])
        group = [current_line]
        remaining_lines = []
        for line in lines:
            if (min(line.dxf.start[1], line.dxf.end[1]) >= span_start and max(line.dxf.start[1], line.dxf.end[1]) <= span_end) or \
               (min(line.dxf.start[1], line.dxf.end[1]) <= span_start and max(line.dxf.start[1], line.dxf.end[1]) >= span_start) or \
               (min(line.dxf.start[1], line.dxf.end[1]) <= span_end and max(line.dxf.start[1], line.dxf.end[1]) >= span_end):
                if 50 < abs(current_line.dxf.start[0] - line.dxf.start[0]) < distance_threshold:
                    group.append(line)
                else:
                    remaining_lines.append(line)
            else:
                remaining_lines.append(line)
        groups.append(group)
        
        # Check if the group has only one line
        if len(group) == 1:
            single_line = group[0]
            for line in lines:
                if 50 < abs(single_line.dxf.start[0] - line.dxf.start[0]) < distance_threshold:
                    group.append(line)
                    remaining_lines.remove(line)  # Remove line from remaining lines
            groups.pop()  # Remove the single-line group
        
        lines = remaining_lines
    
    return groups

def group_Hlines(lines, distance_threshold):
    # Sort the lines by length from longest to shortest
    lines = sorted(lines, key=lambda line: abs(line.dxf.start[0] - line.dxf.end[0]), reverse=True)
    groups = []
    
    while len(lines) > 0:
        current_line = lines.pop(0)
        span_start = min(current_line.dxf.start[0], current_line.dxf.end[0])
        span_end = max(current_line.dxf.start[0], current_line.dxf.end[0])
        group = [current_line]
        remaining_lines = []
        for line in lines:
            if (min(line.dxf.start[0], line.dxf.end[0]) >= span_start and max(line.dxf.start[0], line.dxf.end[0]) <= span_end) or \
               (min(line.dxf.start[0], line.dxf.end[0]) <= span_start and max(line.dxf.start[0], line.dxf.end[0]) >= span_start) or \
               (min(line.dxf.start[0], line.dxf.end[0]) <= span_end and max(line.dxf.start[0], line.dxf.end[0]) >= span_end):
                if 50 < abs(current_line.dxf.start[1] - line.dxf.start[1]) < distance_threshold:
                    group.append(line)
                else:
                    remaining_lines.append(line)
            else:
                remaining_lines.append(line)
        groups.append(group)
        
        # Check if the group has only one line
        if len(group) == 1:
            single_line = group[0]
            for line in lines:
                if 50 < abs(single_line.dxf.start[1] - line.dxf.start[1]) < distance_threshold:
                    group.append(line)
                    remaining_lines.remove(line)  # Remove line from remaining lines
            groups.pop()  # Remove the single-line group
        
        lines = remaining_lines
    
    return groups

def find_walls(file_path, layer_name, threshold):
    horizontal_walls = []
    vertical_walls = []
    dwg = ezdxf.readfile(file_path)
    modelspace = dwg.modelspace()

    # Get all lines in the specified layer
    lines = list(modelspace.query(f'LINE[layer=="{layer_name}"]'))

    # Separate lines into horizontal and vertical
    horizontal_lines = [line for line in lines if abs(line.dxf.start[0] - line.dxf.end[0]) > abs(line.dxf.start[1] - line.dxf.end[1])]
    vertical_lines = [line for line in lines if line not in horizontal_lines]

    print("Total number of lines:", len(lines))
    print("\nNumber of horizontal lines:", len(horizontal_lines))
    print("Number of vertical lines:", len(vertical_lines))

    # Discard lines shorter than threshold
    horizontal_lines = separate_lines(horizontal_lines, threshold)
    vertical_lines = separate_lines(vertical_lines, threshold)

    print("\nNumber of horizontal lines after filtering:", len(horizontal_lines))
    print("Number of vertical lines after filtering:", len(vertical_lines))

    # Group horizontal lines
    horizontal_groups = group_Hlines(horizontal_lines, threshold)

    print("\nNumber of horizontal groups:", len(horizontal_groups))
    for i, group in enumerate(horizontal_groups, start=1):
        print(f"Horizontal Group {i}: {len(group)} lines")

    # Group vertical lines
    vertical_groups = group_Vlines(vertical_lines, threshold)

    print("\nNumber of vertical groups:", len(vertical_groups))
    for i, group in enumerate(vertical_groups, start=1):
        print(f"Vertical Group {i}: {len(group)} lines")

    # Pick the longest line from each group to represent a wall
    for group in horizontal_groups:
        horizontal_walls.append(max(group, key=get_line_length))

    for group in vertical_groups:
        vertical_walls.append(max(group, key=get_line_length))

    return horizontal_walls, vertical_walls

def main4():
    file_path = "/Users/user/Downloads/new-sample.dxf"
    layer_name = "A-WALL"
    threshold = 250  # Threshold for discarding short lines

    # Find horizontal and vertical walls
    horizontal_walls, vertical_walls = find_walls(file_path, layer_name, threshold)

    # Output horizontal walls
    print("\nHorizontal Walls:")
    for i, wall in enumerate(horizontal_walls, start=1):
        print(f"Wall {i}: Length = {get_line_length(wall):.2f}mm")

    # Output vertical walls
    print("\nVertical Walls:")
    for i, wall in enumerate(vertical_walls, start=1):
        print(f"Wall {i}: Length = {get_line_length(wall):.2f}mm")

    # Output total number of walls
    total_walls = len(horizontal_walls) + len(vertical_walls)
    print(f"\nTotal number of walls: {total_walls}")

In [4]:
main4()

Total number of lines: 48

Number of horizontal lines: 24
Number of vertical lines: 24

Number of horizontal lines after filtering: 18
Number of vertical lines after filtering: 20

Number of horizontal groups: 7
Horizontal Group 1: 4 lines
Horizontal Group 2: 3 lines
Horizontal Group 3: 3 lines
Horizontal Group 4: 2 lines
Horizontal Group 5: 2 lines
Horizontal Group 6: 2 lines
Horizontal Group 7: 2 lines

Number of vertical groups: 8
Vertical Group 1: 3 lines
Vertical Group 2: 3 lines
Vertical Group 3: 2 lines
Vertical Group 4: 2 lines
Vertical Group 5: 3 lines
Vertical Group 6: 2 lines
Vertical Group 7: 2 lines
Vertical Group 8: 2 lines

Horizontal Walls:
Wall 1: Length = 30225.00mm
Wall 2: Length = 26225.00mm
Wall 3: Length = 8930.00mm
Wall 4: Length = 7675.00mm
Wall 5: Length = 4930.00mm
Wall 6: Length = 3205.00mm
Wall 7: Length = 2980.00mm

Vertical Walls:
Wall 1: Length = 15225.00mm
Wall 2: Length = 8075.00mm
Wall 3: Length = 7385.00mm
Wall 4: Length = 5225.00mm
Wall 5: Length = 4

In [9]:
# Draw the dxf
import ezdxf

import matplotlib.pyplot as plt

def draw_layer_as_png(file_path, layer_name, output_png):
    # Read the DXF file
    doc = ezdxf.readfile(file_path)

    # Extract entities in the specified layer
    entities = doc.modelspace().query(f'LINE[layer=="{layer_name}"]')

    # Extract coordinates from entities
    coordinates = []
    for entity in entities:
        start_point = (entity.dxf.start[0], entity.dxf.start[1])
        end_point = (entity.dxf.end[0], entity.dxf.end[1])
        coordinates.append({'start_point': start_point, 'end_point': end_point})

    # Plot the entities
    fig, ax = plt.subplots()
    for coord in coordinates:
        start_point = coord['start_point']
        end_point = coord['end_point']
        ax.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color='black')

    # Adjust plot settings
    ax.set_aspect('equal', adjustable='box')
    ax.axis('off')  # Hide axis

    # Save plot as PNG image
    plt.savefig(output_png, bbox_inches='tight', pad_inches=0)
    plt.close()  # Close the plot to release resources

In [11]:
draw_layer_as_png("/Users/user/Downloads/new-sample.dxf", 'A-WALL', 'output_png')

In [1]:
def get_horizontal_and_vertical_lengths(file_path, layer_name):
    horizontal_lengths = []
    vertical_lengths = []
    
    # Read the DXF file
    doc = ezdxf.readfile(file_path)

    # Extract entities in the specified layer
    entities = doc.modelspace().query(f'LINE[layer=="{layer_name}"]')

    # Extract lengths from entities
    for entity in entities:
        start_point = (entity.dxf.start[0], entity.dxf.start[1])
        end_point = (entity.dxf.end[0], entity.dxf.end[1])
        length = calculate_distance(start_point, end_point)
        
        if start_point[0] == end_point[0]:  # Vertical line
            vertical_lengths.append(length)
        else:  # Horizontal line
            horizontal_lengths.append(length)

    # Sort the lengths lists from longest to shortest
    horizontal_lengths.sort(reverse=True)
    vertical_lengths.sort(reverse=True)

    return horizontal_lengths, vertical_lengths

In [ ]:
get_horizontal_and_vertical_lengths("/Users/user/Downloads/new-sample.dxf", 'A-WALL')